# Homework 2: Word Similarity

Student Name: Nodens F. Koren

Student ID: 1060811

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


###
# Your answer BEGINS HERE
###

# Filter out non-alphabetic words and lemmatize the lowercased words.
# Remove duplicates for each paragraph by using the set data structure.
brown_corpus = []
for paragraph in brown.paras():
    word_set = set()
    for sentence in paragraph:
        for word in sentence:
            if word.isalpha():
                lemma = lemmatize(word.lower())
                word_set.add(lemma)
    brown_corpus.append(word_set)

# Reads in the raw set1.tab file and only stores the first three columns which
# we will be using.
unfiltered_dict = {}
set1 = open("set1.tab")
set1.readline()
for line in set1:
    line = line.strip('\n')
    items = line.split('\t')
    k, v = (items[0], items[1]), float(items[2])
    unfiltered_dict[k] = v

# This helper function calculates the document frequency for each word from 
# set1.tab in the brown_corpus.
def get_document_frequency(word, corpus):
    count = 0
    for document in corpus:
        if word in document:
            count += 1
    return count

# Filter out any word (key) pairs where at least one of the two words 
# has a document frequency of  <8  in the brown_corpus.
filtered_dictionary = {}
for k, v in unfiltered_dict.items():
    if get_document_frequency(k[0], brown_corpus) >= 8 and get_document_frequency(k[1], brown_corpus) >= 8:
        filtered_gold_standard[k] = v
            
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('footbal

<b>For your testing: </b>

In [0]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [3]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###

# This helper function checks whether the given word 
# (a) has only one sense (i.e. only one synset), or 
# (b) where the count (as provided by the WordNet count() method for the lemmas
#     associated with a synset) of the most common sense is at least 4 times 
#     larger than the next most common sense.
# (c) has noun as its primary sence
def is_single_primarysense(word):

    # If the given word has only 1 sense
    if len(wordnet.synsets(word)) == 1:
        # Check if this sense is a noun, if so consider it valid.
        if wordnet.synsets(word)[0].pos() == 'n':
            word_primarysense[word] = wordnet.synsets(word)[0]
            return True
        else:
            return False

    # If the given word has multiple senses    
    else:
        # Count the number of lemma associated with each sense
        counts = []
        for synset in wordnet.synsets(word):
            count = 0
            for lemma in synset.lemmas():
                # Consider only the word's own lemma, both the upper and the
                # lower cases.
                if lemma.name().lower() == word:
                    count += lemma.count()
            counts.append(count)
        primarysense = wordnet.synsets(word)[counts.index(max(counts))] 
        counts.sort(reverse=True)

        # Check if the primary sense has a count at least 4 times greater
        # than other senses. 
        if counts[0] >= (4 * counts[1]) and counts[0] != 0:
            # If so, check if this sense is a noun, if so consider it valid.
            if primarysense.pos() == 'n':
                word_primarysense[word] = primarysense
                return True
            else:
                return False
        else:
            return False

final_dict = {}

# Filter out any word (key) pairs where at least one of the two words 
# does not have single primary sense.
for (k, v) in filtered_gold_standard.items():
    if is_single_primarysense(k[0]) and is_single_primarysense(k[1]):
        final_gold_standard[k] = v

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [0]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [5]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

# Calculate and store the lin similarity for each word pair in final_gold_standard
for k in final_gold_standard.keys():
    v = word_primarysense[k[0]].lin_similarity(word_primarysense[k[1]], brown_ic)
    lin_similarities[k] = v

###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.
{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.060049798869

<b>For your testing:</b>

In [0]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [11]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math

# This helper function calculates the NPMI similarity between a word pair based
# on the brown_corpus.
def calculate_NPMI(x, y):
    x_count = 0
    y_count = 0
    joint_count = 0
    total_count = 0
    for paragraph in brown_corpus:
        # Calculate the number of word types in the current paragraph
        total_count += len(paragraph)

        # Check the existence of word x in the current paragraph
        if x in paragraph:
            x_count += 1

        # Check the existence of word y in the current paragraph                   
        if y in paragraph:
            y_count += 1

        # Calculate the co-occurence of word x and y in the current paragraph
        if x in paragraph and y in paragraph:
            joint_count += 1
    
    if joint_count == 0:
        return -1
    return (math.log2((x_count/total_count) * (y_count/total_count)) / math.log2(joint_count/total_count)) - 1

# Calculate and store the NPMI similarity for each word pair in final_gold_standard
for k in final_gold_standard.keys():
    v = calculate_NPMI(k[0], k[1])
    NPMI_similarities[k] = v

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6531272737497535, ('professor', 'doctor'): -1, ('student', 'professor'): 0.535495995400926, ('stock', 'egg'): 0.3685128922213745, ('money', 'cash'): 0.44493834723517245, ('king', 'queen'): 0.4181407297139583, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254175, ('baby', 'mother'): 0.5149353890388502, ('car', 'automobile'): 0.5430334549802616, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.5861510629843374, ('brother', 'monk'): 0.42993185256931743, ('journey', 'car'): -1, ('coast', 'hill'): 0.33931028834846244, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892886, ('psychology', 'doctor'): 0.4613281583726412, ('psychology', 'mind'): 0.4461666043435182, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534864, ('planet', 'moon'): 0.658000616232332, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [0]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [18]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.spatial.distance import cosine as cos_distance

vectorizer = DictVectorizer()
svd = TruncatedSVD(n_components=500, random_state=1)

# Build a document-term frequency matrix based on the brown_corpus
brown_matrix = vectorizer.fit_transform([dict.fromkeys(paragraph, 1) for paragraph in brown_corpus])
# Transpose the document-term frequency matrix to a term-document frequency 
# matrix and truncate each row of the document-term frequency matrix to dense 
# vectors of length k = 500.
brown_matrix = svd.fit_transform(brown_matrix.transpose())

# Calculate and store the LSA similarity for each word pair in final_gold_standard
for k in final_gold_standard.keys():
    vector_1, vector_2 = vectorizer.transform({k[0]:1}), vectorizer.transform({k[1]:1})
    LSA_similarities[k] = 1 - cos_distance(vector_1 * brown_matrix, vector_2 * brown_matrix)

###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.3152754702572451, ('professor', 'doctor'): 0.05844298794179126, ('student', 'professor'): 0.2767337329493138, ('stock', 'egg'): 0.14436308551189037, ('money', 'cash'): 0.15554334593699293, ('king', 'queen'): 0.1302217523561776, ('bishop', 'rabbi'): 0.031084205148395827, ('football', 'basketball'): 0.24386084983005551, ('football', 'tennis'): 0.12632217011903002, ('alcohol', 'chemistry'): 0.07876794799305287, ('baby', 'mother'): 0.35791140240818753, ('car', 'automobile'): 0.3600832277825228, ('journey', 'voyage'): 0.10533899235161615, ('coast', 'shore'): 0.40214077624750144, ('brother', 'monk'): 0.11876162813444846, ('journey', 'car'): 0.001960165045462592, ('coast', 'hill'): 0.22113053499136226, ('forest', 'graveyard'): 0.08874563656026124, ('monk', 'slave'): -0.030531133573225677, ('coast', 'forest'): 0.13886784050556522, ('psychology', 'doctor'): 0.14332368812525242, ('psychology', 'mind'): 0.13327874870626477, ('psychology', 'health'): 0.027449699898818847, (

<b>For your testing:</b>

In [0]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [20]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])
word_dict = {}
###
# Your answer BEGINS HERE
###

# Collect a dictionary of lowercased words and their number of occurrences in
# the original Brown corpus.
for paragraph in brown.paras()[0:12000]:
    for sentence in paragraph:
        for word in sentence:
            if word.lower() in word_dict.keys():
                word_dict[word.lower()] += 1
            else:
                word_dict[word.lower()] = 1        

# Filter out words which appears less than 4 times and store the rest to the
# vocabulary set.
for k, v in word_dict.items():
    if v >= 5:
        vocab.add(k)
                
###
# Your answer ENDS HERE
###

print(len(vocab))

12681


**For your testing:**

In [0]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [22]:
###
# Your answer BEGINS HERE
###
import numpy as np
from numpy import array
from numpy import argmax
from itertools import tee

# Build an index table for each word in vocab
word2idx = {}
i = 0
for word in vocab:
    word2idx[word] = i
    i += 1

# This helper function iterates over each sentence in a 
# (w0, w1, w2)->(w1, w2, w3)->(w2, w3, w4)... manner.
def triwise(iterable):
    a, b = tee(iterable)
    a, c = tee(iterable)
    next(b, None)
    next(c, None)
    next(c, None)
    return zip(a, b, c)

x_train = []
y_train = []
x_dev = []
y_dev = []

# Construct the training set
for paragraph in brown.paras()[:12000]:
    for sentence in paragraph:
        # Ignores sentences with length < 3.
        if len(sentence) < 3:
            continue
        else:
            for a, b, c in triwise(sentence):
                a, b, c = a.lower(), b.lower(), c.lower()
                # If the word is not in vocab, converts it to <UNK> instead.
                train_1 = word2idx[a] if a in vocab else word2idx["<UNK>"]
                train_2 = word2idx[b] if b in vocab else word2idx["<UNK>"]
                target = word2idx[c] if c in vocab else word2idx["<UNK>"]
                x_train.append((train_1, train_2))
                y_train.append(target)

# Construct the development set
for paragraph in brown.paras()[12000:]:
    for sentence in paragraph:
        if len(sentence) < 3:
            continue      
        else:
            for a, b, c in triwise(sentence):
                a, b, c = a.lower(), b.lower(), c.lower()
                # If the word is not in vocab, converts it to <UNK> instead.                
                dev_1 = word2idx[a] if a in vocab else word2idx["<UNK>"]
                dev_2 = word2idx[b] if b in vocab else word2idx["<UNK>"]
                target = word2idx[c] if c in vocab else word2idx["<UNK>"]
                x_dev.append((dev_1, dev_2))
                y_dev.append(target)
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

###
# Your answer ENDS HERE
###

print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823,)
(174016, 2)
(174016,)


**For your testing:**

In [0]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [24]:
lm_similarities = {}
from keras.layers import Dense, Input, Embedding, Flatten
from keras.models import Model
from keras.optimizers import Adam

###
# Your answer BEGINS HERE
###

# Declare a embedding layer with the input dimension being the number of
# words in vocab and the output dimension being 100. The trainable option
# is set to be on.
embedding_layer = Embedding(
    len(vocab),
    100,
    trainable=True
)

# Set the input shape to be 2 to take in the bi-gram inputs.
input_ = Input(shape=(2,))
x = embedding_layer(input_)

# Concatenate the word embedding for each word in the bi-gram pair.
x = Flatten()(x)

# The h=tanh(W1x′+b) layer with 100 units
x = Dense(100, activation='tanh')(x)

# Output layer
x = Dense(len(list(vocab)), activation='softmax')(x)

model = Model(inputs=input_, output=x)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy', 'cosine_similarity']
)

# Train the model for 3 epochs with a batch size of 256.
r = model.fit(
  x_train,
  y_train,
  batch_size=256,
  epochs=3,
  validation_data = (x_dev, y_dev)
)

# Extract the word embedding weight
embeddings = embedding_layer.get_weights()[0]
words_embeddings = {w: embeddings[idx] for w, idx in word2idx.items()}

# Calculate the cosine similarity for each pair of words in the final_gold_standard
# based on the trained word embedding vectors
for k in final_gold_standard.keys():
    word1 = k[0]
    word2 = k[1]
    if k[0] not in vocab:
        word1 = "<UNK>"
    if k[1] not in vocab:
        word2 = "<UNK>"
    lm_similarities[k] = 1 - cos_distance(words_embeddings[word1], words_embeddings[word2])

###
# Your answer ENDS HERE
###


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 872823 samples, validate on 174016 samples
Epoch 1/3
872823/872823 [==============================] - 30s 34us/step - loss: 5.9388 - accuracy: 0.1496 - cosine_similarity: 9.0919 - val_loss: 5.2474 - val_accuracy: 0.1667 - val_cosine_similarity: 5.9600
Epoch 2/3
872823/872823 [==============================] - 29s 33us/step - loss: 5.4313 - accuracy: 0.1839 - cosine_similarity: 5.8725 - val_loss: 5.1332 - val_accuracy: 0.1727 - val_cosine_similarity: 5.3007
Epoch 3/3
872823/872823 [==============================] - 29s 33us/step - loss: 5.2302 - accuracy: 0.1955 - cosine_similarity: 5.6881 - val_loss: 5.0848 - val_accuracy: 0.1770 - val_cosine_similarity: 5.1381


In [25]:
print(lm_similarities)

{('bread', 'butter'): 0.23464332520961761, ('professor', 'doctor'): 0.42521747946739197, ('student', 'professor'): 0.5182636976242065, ('stock', 'egg'): 0.2642689645290375, ('money', 'cash'): 0.24615910649299622, ('king', 'queen'): 0.5198251008987427, ('bishop', 'rabbi'): 0.30148082971572876, ('football', 'basketball'): 0.4315166771411896, ('football', 'tennis'): 0.38201990723609924, ('alcohol', 'chemistry'): 0.06805875152349472, ('baby', 'mother'): 0.3200019896030426, ('car', 'automobile'): 0.011390027590095997, ('journey', 'voyage'): 0.3481979966163635, ('coast', 'shore'): 0.055063918232917786, ('brother', 'monk'): 0.3916759192943573, ('journey', 'car'): 0.3893755078315735, ('coast', 'hill'): 0.2864517867565155, ('forest', 'graveyard'): -0.12595079839229584, ('monk', 'slave'): -0.057303737848997116, ('coast', 'forest'): 0.5638205409049988, ('psychology', 'doctor'): 0.35006019473075867, ('psychology', 'mind'): 0.07453189045190811, ('psychology', 'health'): 0.3338579833507538, ('psycho

## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [26]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
lin_list = []
NPMI_list = []
LSA_list = []
lm_list = []
final_gold_standard_list = []

# Sort the vector for each similarity dictionaries so that they are in the
# same order.
for k in final_gold_standard:
    lin_list.append(lin_similarities[k])
    NPMI_list.append(NPMI_similarities[k])
    LSA_list.append(LSA_similarities[k])
    lm_list.append(lm_similarities[k])
    final_gold_standard_list.append(final_gold_standard[k])

# Calculate the pearson_correlations between each similarity vector and the
# final_gold_standard_list.
pearson_correlations['lin'] = pearsonr(lin_list, final_gold_standard_list)[0]
pearson_correlations['NPMI'] = pearsonr(NPMI_list, final_gold_standard_list)[0]
pearson_correlations['LSA'] = pearsonr(LSA_list, final_gold_standard_list)[0]
pearson_correlations['lm'] = pearsonr(lm_list, final_gold_standard_list)[0]
    
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.18799888869410047, 'LSA': 0.3888687679346686, 'lm': 0.19255380954350473}


<b>For your testing:</b>

In [0]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.